<img src="https://github.com/gantian127/bmi_roms/blob/master/docs/source/_static/logo.png?raw=true" width='600' align='center'></a>

# Quick Start Tutorial 

This notebook will demonstrate how to use the bmi_roms package to download the [ROMS model](https://www.myroms.org/) datasets. 

This tutorial includes the following sections:

1. [Brief Introduction](#section1)

   This section provides basic information about bmi_roms package. 
   <br>
   
2. [Start with Examples](#section2)
   
   This section provides an example to demonstrate how to use the bmi_roms package to download the ROMS model output dataset for visualization.
   <br>
   


**Suggested citation**: Gan, T. (2023). Jupyter Notebooks for the ROMS Data Component, HydroShare, https://www.hydroshare.org/resource/5bed8401cfe04c38b0f84119b1999482/

<a id='section1'></a>
## 1. Brief Introduction

bmi_roms package is an implementation of the Basic Model Interface ([BMI](https://bmi.readthedocs.io/en/latest/)) for the [ROMS model](https://www.myroms.org/) dataset. This package is not implemented for people to use but is the key element to convert the ROMS output dataset into a data component for the [PyMT](https://pymt.readthedocs.io/en/latest/?badge=latest) modeling framework developed by Community Surface Dynamics Modeling System ([CSDMS](https://csdms.colorado.edu/wiki/Main_Page)). 

The current implementation supports 2D, 3D and 4D ROMS output datasets defined with geospatial and/or time dimensions (e.g., dataset defined with dimensions as [time, s_rho, eta_rho, xi_rho] )

To install bmi_roms package, please follow the instructions [here](https://github.com/gantian127/bmi_roms#install-package).

<a id='section2'></a>
## 2. Code Example

This section provides an example to demonstrate how to use the bmi_roms package to access and load the ROMS dataset for visualization.

Import BmiRoms class and instantiate it. A configuration file (yaml file) is required to provide the parameter settings. An example config_file.yaml file is provided in the same folder with this Jupyter Notebook. 

In [ ]:
from bmi_roms import BmiRoms
import numpy as np
import matplotlib.pyplot as plt

data_comp = BmiRoms()
data_comp.initialize('config_file.yaml')

If you check the content in the config_file.yaml, you will find that the parameter includes a file name to load the local ROMS model dataset. The example.nc file is provided in the same folder with this Jupyter Notebook. You can also set the "filename" value with an OPeNDAP Data URL to access remote dataset and donwload it by setting the "download" parameter as True. Please see more details [here](https://bmi-roms.readthedocs.io/en/latest/?badge=latest#parameters) for the parameter settings.

In [ ]:
!cat config_file.yaml

Use variable related methods from BmiRoms class to check the variable information. 

In [ ]:
# get variable info
for var_name in  data_comp.get_output_var_names():
    var_unit = data_comp.get_var_units(var_name)
    var_location = data_comp.get_var_location(var_name)
    var_type = data_comp.get_var_type(var_name)
    var_grid = data_comp.get_var_grid(var_name)
    var_itemsize = data_comp.get_var_itemsize(var_name)
    var_nbytes = data_comp.get_var_nbytes(var_name)
    print('variable_name: {} \nvar_unit: {} \nvar_location: {} \nvar_type: {} \nvar_grid: {} \nvar_itemsize: {}' 
            '\nvar_nbytes: {} \n'. format(var_name, var_unit, var_location, var_type, var_grid, var_itemsize, var_nbytes))


Use time related methods from BmiRoms class to check the time information. The time values are stored in a format which follows [CF convention](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.8/cf-conventions.pdf).

In [ ]:
# get time info
start_time = data_comp.get_start_time()
end_time = data_comp.get_end_time()
time_step = data_comp.get_time_step()
time_unit = data_comp.get_time_units()
time_steps = int((end_time - start_time)/time_step) + 1
print('start_time:{} \nend_time:{} \ntime_step:{} \ntime_unit:{} \ntime_steps:{} \n'.format(
    start_time, end_time, time_step, time_unit, time_steps))

Use grid related methods of BmiRoms class to check the grid information. You will find out that the variables are defined on different grids.

In [ ]:
# get variable grid info 
for var_name in data_comp.get_output_var_names():
    var_grid = data_comp.get_var_grid(var_name)
    
    grid_rank = data_comp.get_grid_rank(var_grid) 
    grid_size = data_comp.get_grid_size(var_grid)
    
    grid_shape = np.empty(grid_rank, int)
    data_comp.get_grid_shape(var_grid, grid_shape)
    
    grid_spacing = np.empty(grid_rank)
    data_comp.get_grid_spacing(var_grid, grid_spacing)
    
    grid_origin = np.empty(grid_rank)
    data_comp.get_grid_origin(var_grid, grid_origin)
    
    print('var_name: {} \ngrid_id: {}\ngrid_rank: {} \ngrid_size: {} \ngrid_shape: {} \ngrid_spacing: {} \ngrid_origin: {} \n'.format(
        var_name, var_grid, grid_rank, grid_size, grid_shape, grid_spacing, grid_origin))

Use get_value( ) method to get the variable data as a numpy array. Please note that the get_value( ) method is to flatten data arrays. So make a new variable that restores the dimensionality of the data.

The code below shows how to get the variable values for the 'time-averaged salinity' and make a contour plot. The latitude and longitude values can be obtained from the 'latitude of RHO-points' and 'longitude of RHO-points' variables.

In [ ]:
# get variable data 
data = np.empty(1026080, 'float64')
data_comp.get_value('time-averaged salinity', data)
data_3D = data.reshape([40, 106, 242])

# get lon and lat data
lat = np.empty(25652, 'float64')
data_comp.get_value('latitude of RHO-points', lat)

lon = np.empty(25652, 'float64')
data_comp.get_value('longitude of RHO-points', lon)

In [ ]:
# make a contour plot
fig = plt.figure(figsize=(10,7))
im = plt.contourf(lon.reshape([106, 242]), lat.reshape([106, 242]), data_3D[0], levels=36)
fig.colorbar(im)
plt.axis('equal')
plt.xlabel('Longitude [degree_east]')
plt.ylabel('Latitude [degree_north]')
plt.title('ROMS model data of time-averaged salinity')

Complete the example by finalizing the component. finalize( ) method performs tasks that take place after using the data component, such as deallocating memory and closing files.

In [ ]:
data_comp.finalize()